In [1]:
%pip install -q -U google-generativeai
%pip install google-cloud-aiplatform
%pip install --upgrade -q google-api-python-client google-auth-httplib2 google-auth-oauthlib
%pip install mediapipe

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.20.0 requires protobuf<4,>=3.12, but you have protobuf 4.25.5 which is incompatible.
mediapipe 0.10.11 requires protobuf<4,>=3.11, but you have protobuf 4.25.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached protobuf-3.20.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
g

In [2]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import google.generativeai as genai

SCOPES = ['https://www.googleapis.com/auth/generative-language.retriever']

def load_creds():
    """Converts `client_secret.json` to a credential object.

    This function caches the generated tokens to minimize the use of the
    consent screen.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

creds = load_creds()

genai.configure(credentials=creds)

In [7]:
import csv
from collections import defaultdict

def get_labels():
    labels = defaultdict(list)
    
    with open('foreground_labels_small.csv', 'r') as file:
        csv_reader = csv.reader(file)
        # print(csv_reader.to_list())
        first_row = True
        for row in csv_reader:
            if first_row:
                first_row = False
                continue
            # print(row)
            labels[row[1]].append(int(row[0]))
            # break
    return labels          
            
    # print(list(labels))
output = get_labels()
print(list(output.keys()))
print(output)

['Cat', 'Bird', 'Equestrian', 'Dog', 'Person', 'Woman', 'Lamb', 'Plant', 'Baby', 'Foal', 'Birds', 'Martial Arts', 'Planter', 'Horse', 'Chicken', 'Sports', 'Cattle', 'Sheep', 'Cycling', 'Portrait', 'People', 'Man', 'Animal', 'Musician', 'Child', 'Horses', 'Beach', 'Cow', 'Agriculture', 'Sculpture', 'Livestock', 'Toddler', 'Motorcycle', 'Plants', 'Girl', 'Owl', 'Police', 'Children', 'Puppy', 'Elephant', 'Aviation', 'Horseback riding', 'Flamingo', 'Parking', 'Boy', 'Flowers', 'Medical', 'Fashion']
defaultdict(<class 'list'>, {'Cat': [0, 6, 9, 11, 17, 58, 67, 69, 70, 77, 84, 90, 93, 103, 106, 131, 142, 143, 153, 173, 193, 247, 265, 274, 277, 286, 325, 326, 332, 345, 356, 361, 365, 369, 375, 404, 410, 413, 421, 426, 434, 435, 439, 448, 460, 461, 472, 477, 481, 482, 486, 490, 491, 493, 494, 496, 500], 'Bird': [1, 4, 16, 19, 29, 30, 32, 37, 38, 41, 42, 45, 46, 50, 53, 59, 71, 72, 80, 83, 99, 109, 112, 113, 118, 125, 130, 133, 149, 154, 160, 161, 163, 164, 168, 169, 170, 177, 181, 186, 199, 20

In [5]:
# take in a background image, and return what foreground image to occlude
import PIL.Image
import google.generativeai as genai


def fos():
    labels = get_labels()
    label_titles = list(labels.keys())
    # n02924116_10146.JPEG
    skier_image = PIL.Image.open('/srv/PASCAL3D+_release1.1/Images/bus_imagenet/n02924116_10146.JPEG')
    print(skier_image)
    
    model = genai.GenerativeModel("gemini-1.5-pro")
    # # prompt = "The goal is for you to go through a list of foreground objects, and select which one is the most realistic to cover the object in the image provided. The list of foreground objects are: [car, cat, dog, bird, bus, baby, tree]. You are forced to choose an option, and it should be a one word answer."
    # # prompt = "You will be a given a list of foreground objects, and you are suppsoe to choose a category from that list to cover the subject in the image provided. Your answer should be one category from the list provided and should be a one word answer. You have to make a selection, and cannot say no. The list is [car, cat, dog, bird, bus, baby, tree]."
    prompt = "You will be a given a list of foreground objects, and you are suppsoe to choose a category from that list to cover the subject in the image provided. Your answer should be one category from the list provided and should be a one word answer. You have to make a selection, and cannot say no. The list is: " + str(label_titles)
    print(prompt)
    # response = model.generate_content([prompt, skier_image])
    # print(response)

print(fos())

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x332 at 0x7F4858FE1AC0>


AttributeError: module 'google.generativeai' has no attribute 'GenerativeModel'

In [1]:
import os

'''
Important paths
'''
path_to_original_pascal3dp = '/srv/PASCAL3D+_release1.1/'

small_occluder_path = '/srv/occluder_libs_test_small.npz'
medium_occluder_path = '/srv/occluder_libs_test_medium.npz'
large_occluder_path = '/srv/occluder_libs_test_large.npz'

# just for now...
occluder_path = '/srv/occluder_libs_test_low.npz'

bg_list_path = path_to_original_pascal3dp + 'Image_sets/%s_imagenet_val.txt'
bg_img_path = path_to_original_pascal3dp + 'Images/%s_imagenet'
bg_anno_path = path_to_original_pascal3dp + 'Annotations/%s_imagenet'
bg_mask_path = path_to_original_pascal3dp + 'obj_mask/%s'

path_save = './results'
save_img_path = path_save + '/images'
save_anno_path = path_save + '/annotations'

In [26]:
categories = ['bicycle', 'bus', 'car', 'motorbike', 'train']

occlusion_ranges = {
    "low": (0.2, 0.4),
    "medium": (0.4, 0.6),
    "high": (0.6, 0.8)
}

dataset = {"low": [], "medium": [], "high": []}

In [27]:
import scipy

def load_one_annotation(anno_path):
    a = scipy.io.loadmat(anno_path)
    # I added the astype int here....
    bbox_ = a['record'][0][0][1][0][0][1][0].astype(int)
    w = a['record']['size'][0][0][0][0][0]
    h = a['record']['size'][0][0][0][0][1]
    num_obj = len(a['record'][0][0][1][0])
    return w, h, bbox_, num_obj != 1

In [28]:
'''
get overlap percentage based on bounding boxes
'''
def overlap_ratio(occluder_bb, occludee_bb):

    #top left and bottom right points
    occluder_x1, occluder_y1, occluder_x2, occluder_y2 = occluder_bb
    occludee_x1, occludee_y1, occludee_x2, occludee_y2 = occludee_bb

    # area of the foreground object
    occludee_area = (occludee_x2 - occludee_x1) * (occludee_y2 - occludee_y1)

    # area of the background object being covered by the foreground object
    overlap_area = max(0, min(occludee_x2, occluder_x2) - max(occludee_x1, occluder_x1)) * max(0, min(occludee_y2, occluder_y2) - max(occludee_y1, occluder_y1))

    # overlap over the total background object area
    return overlap_area / occludee_area

In [29]:
import random
import time

'''
Get a list of randomly chosen bounding boxes to occlude the background object above some threshold
This can be improved if we know the foreground image has to be some base scale to allow for above threshold occlusion
'''
def get_bbox_list(bg_bbox, fg_mask, bg_w, bg_h, fg_w, fg_h):

    random.seed(time.time())

    bboxes = []
    occludee_x1 = bg_bbox[0]
    occludee_y1 = bg_bbox[1] 
    occludee_x2 = bg_bbox[2]
    occludee_y2 = bg_bbox[3]
    num_boxes = 20
    
    for _ in range(num_boxes):

        occluder_x1 = random.randint(max(0, occludee_x1 - fg_w), occludee_x2) # overlapping in the x-direction
        occluder_y1 = random.randint(max(0, occludee_y1 - fg_h), occludee_y2) # overlapping in the y-direction

        # TODO: Fix so doesnt exceed background image
        
        occluder_x2 = occluder_x1 + fg_w
        occluder_y2 = occluder_y1 + fg_h

        if occluder_x2 > bg_w or occluder_y2 > bg_h:
            continue

        occluder_bb = [occluder_x1, occluder_y1, occluder_x2, occluder_y2]
        occluded_ratio = overlap_ratio(occluder_bb, bg_bbox)

        if occluded_ratio >= .20:
            bboxes.append((occluded_ratio, [occluder_x1, occluder_y1, occluder_x2, occluder_y2]))
    
    return bboxes

In [30]:
'''
returns the score, composite image, and compositive mask. 
num scales is the number of different foreground scales to try.
'''
from libcom import OPAScoreModel

def get_optimal_location(fg_img, fg_mask, bg_img, bg_w, bg_h, bg_bbox, num_scales):

    net = OPAScoreModel(device=0, model_type='SimOPA')
    cache_dir = './cache'

    # from libcom.fopa_heat_map.source.prepare_multi_fg_scales import prepare_multi_fg_scales
    scaled_fg_dir, scaled_mask_dir, csv_path = prepare_multi_fg_scales(cache_dir, fg_img, fg_mask, bg_img, 16)

    score = 0
    ratio = 0
    optimal_bbox = None
    best_fg = None
    best_mask = None
    best_comp = None 
    best_comp_mask = None
    
    with open(csv_path, mode='r', newline='') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            fg_name   = '{}_{}_{}_{}.jpg'.format(row["fg_name"].split(".")[0],row["bg_name"].split(".")[0],int(row["newWidth"]),int(row["newHeight"]))
            mask_name = '{}_{}_{}_{}.jpg'.format(row["fg_name"].split(".")[0],row["bg_name"].split(".")[0],int(row["newWidth"]),int(row["newHeight"]))
            fg_w = int(row['newWidth'])
            fg_h = int(row['newHeight'])

            # why are we doing this...
            bg_img    = read_image_pil(bg_img)
            bbox_list = get_bbox_list(bg_bbox, fg_mask, bg_w, bg_h, fg_w, fg_h)

            fg_img = os.path.join(scaled_fg_dir, fg_name)
            fg_mask = os.path.join(scaled_mask_dir, mask_name)

            for occ_ratio, bbox in bbox_list:
                comp, comp_mask = get_composite_image(fg_img, fg_mask, bg_img, bbox)
                bbox_score = net(comp, comp_mask)
                if bbox_score > score:
                    best_fg = fg_img
                    best_mask = fg_mask
                    optimal_bbox = bbox
                    best_comp = comp
                    best_comp_mask = comp_mask
                    score = bbox_score
                    ratio = occ_ratio

        return score, ratio, best_fg, best_mask, optimal_bbox, best_comp, best_comp_mask

In [31]:
import random
import numpy as np
from PIL import Image
import cv2
import time

'''
Get a random occluder and correspondiong mask
'''
def get_occluder():
    data = np.load(occluder_path, allow_pickle=True)

    random.seed(time.time())

    os.makedirs('./occluders', exist_ok=True)
    
    i = random.randint(0, len(data['images']))
    occ_img = f'./occluders/fg_img_{i}.jpg'
    occ_mask = f'./occluders/fg_mask_{i}.png'

    image = data['images'][i]
    mask = data['masks'][i]
    mask = (mask * 255).astype(np.uint8)
        
    cv2.imwrite(occ_img, image)
    cv2.imwrite(occ_mask, mask)

    return occ_img, occ_mask

In [ ]:
def fos(

In [44]:
from libcom import color_transfer
from libcom.utils.process_image import *
from libcom.utils.environment import *
from libcom import OPAScoreModel
from libcom import get_composite_image
from libcom.utils.process_image import make_image_grid
import cv2
import csv
from PIL import Image
from libcom.fopa_heat_map.source.prepare_multi_fg_scales import prepare_multi_fg_scales
from libcom import Mure_ObjectStitchModel
from libcom import ControlComModel
import json

def generate_composite_image(cate, bg_img, bg_w, bg_h, bg_bbox, save_img_path):

    net = Mure_ObjectStitchModel(device=0, sampler='plms')

    # maybe a list isnt necessary here? They do one anntotation for each of the occlusion ratios in OccludedPascal3D+
    annotation = {}

    # fg_mask is a path to an image here.... but bg_mask is a numpy array!
    fg_img, fg_mask = get_occluder()

    occluder_path = fg_img

    img_id = bg_img.split('/')[-1].split('.')[0]
    
    score, ratio, fg_img, fg_mask, bbox, comp, comp_mask = get_optimal_location(fg_img, fg_mask, bg_img, bg_w, bg_h, bg_bbox, num_scales=16)

    # x1, y1, x2, y2 = bg_bbox

    annotation['box'] = bg_bbox.tolist() 
    annotation['ratio'] = ratio
    annotation['occluder_box'] = bbox
    annotation['occluder_path'] = occluder_path
    annotation['source'] = bg_img
    annotation['cate'] = cate

    # write to a directory based on the level of occlusion
    occlusion_level = None
    for range_label, (min_occ, max_occ) in occlusion_ranges.items():
        if min_occ <= ratio <= max_occ:
            occlusion_level = range_label
            
    if(score):
        res, show_fg_img = net(bg_img, [fg_img], [fg_mask], bbox, sample_steps=25, num_samples=3)

        # Convert and write JSON object to file
        with open(f'{save_anno_path}/{occlusion_level}/{img_id}.json', "w") as outfile: 
            json.dump(annotation, outfile)
        
        # TODO: find the best result in res, instead of just taking the last sample!!!!
        cv2.imwrite(f'{save_img_path}/{occlusion_level}/{img_id}.jpg', res[2])

In [45]:
#import BboxTools as bbt
import os

# the only reason we have bg_mask_dir is in case we need it for segmentation masks...
def generate_dataset(cate, file_list, bg_img_dir, bg_anno_dir, save_img_dir, save_anno_dir, record_file):

    os.makedirs(save_img_dir, exist_ok=True)
    os.makedirs(save_anno_dir, exist_ok=True)

    # make a different directory for each range
    for range_label in occlusion_ranges.keys():
        os.makedirs(save_img_dir + "/" + range_label, exist_ok=True)
        os.makedirs(save_anno_dir + "/" + range_label, exist_ok=True)
    
    for file_name in file_list[9:10]:
        bg_w, bg_h, bg_bbox, flag_ = load_one_annotation(os.path.join(bg_anno_dir, file_name + '.mat'))

        if flag_:
            record_file.write('Skipped %s for multi objects\n' % file_name)
            continue

        bg_img = os.path.join(bg_img_dir, file_name + '.JPEG')
        # bg_mask = np.array(Image.open(os.path.join(bg_mask_dir, file_name + '.JPEG')))

        #if not bg_mask.shape[0] == bg_img.shape[0] and bg_mask.shape[1] == bg_img.shape[1]:
            #bg_mask = cv2.resize(mask, (bg_img.shape[1], bg_img.shape[0]), interpolation=cv2.INTER_NEAREST)

        # not sure why this is necessary, took from OccludedPascal3D+ code
        #bg_bbox = bbt.from_numpy(bg_bbox, image_boundary=img.shape[0:2], sorts=('x0', 'y0', 'x1', 'y1'))

        generate_composite_image(cate, bg_img, bg_w, bg_h, bg_bbox, save_img_path)

In [46]:
for cate in categories:
    print('Start cate: ', cate)
    tem = open('generating_record_%s_1030.txt' % cate, 'w')
    file_list_ = open(bg_list_path % cate).readlines()
    file_list_ = [tem.strip('\n') for tem in file_list_]
    bg_img_path_ = bg_img_path % cate
    bg_anno_path_ = bg_anno_path % cate

    generate_dataset(cate, file_list_, bg_img_path_, bg_anno_path_, save_img_path, save_anno_path, tem)

Start cate:  bicycle
Start cate:  bus
Start cate:  car
Start cate:  motorbike
Start cate:  train
